In [15]:
import pandas as pd

import utils
import soundfile

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
meta = pd.read_csv('../data/toy_final/metadata.csv')
meta.head()

,index,filepath,label,label_length,spec_length
0,0,../data\toy_raw\174\168635\174-168635-0000.flac,"[8, 5, 0, 8, 1, 4, 0, 14, 5, 22, 5, 18, 0, 2, ...",45,452
1,1,../data\toy_raw\174\168635\174-168635-0001.flac,"[20, 8, 5, 0, 8, 5, 1, 18, 20, 0, 15, 6, 0, 20...",50,464
2,2,../data\toy_raw\174\168635\174-168635-0002.flac,"[8, 9, 19, 0, 19, 9, 19, 20, 5, 18, 0, 1, 14, ...",224,1585
3,3,../data\toy_raw\174\168635\174-168635-0003.flac,"[8, 5, 0, 19, 21, 6, 6, 5, 18, 5, 4, 0, 1, 12,...",177,1317
4,4,../data\toy_raw\174\168635\174-168635-0004.flac,"[15, 14, 12, 25, 0, 1, 19, 0, 8, 5, 0, 23, 1, ...",169,1276


In [13]:
audio_paths = meta.filepath.to_list()
all_label = meta.label.to_list()

In [23]:
au = tf.io.read_file(audio_paths[0])
tf.audio.decode_wav(au)

InvalidArgumentError: Header mismatch: Expected RIFF but found fLaC [Op:DecodeWav]

In [27]:
ds = tf.data.Dataset.from_tensor_slices((audio_paths, all_label))

def load_and_preprocess_audio(path):
    au = tf.io.read_file(path)
    feature = utils.compute_spectrogram_feature(au.numpy(), 16000)
    feature = utils.normalize_audio_feature(feature)
    feature = pad_sequences(feature.T, maxlen= 2000, padding = 'post')
    feature = feature.T 
    return feature
    
# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, label):
    return load_and_preprocess_audio(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)

BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE


# Setting a shuffle buffer size as large as the dataset ensures that the data is
# completely shuffled.
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# prefetch lets the dataset fetch batches in the background while the model is training.
ds = ds.prefetch(buffer_size=AUTOTUNE)

AttributeError: in converted code:

    <ipython-input-24-163aaf7c365e>:13 load_and_preprocess_from_path_label  *
        return load_and_preprocess_audio(path), label
    <ipython-input-27-e3cb3d35c6c1>:5 load_and_preprocess_audio  *
        feature = utils.compute_spectrogram_feature(au.numpy(), 16000)

    AttributeError: 'Tensor' object has no attribute 'numpy'
